In [ ]:
#used https://github.com/carlini/audio_adversarial_examples for the CW attack
#the second part was not finished due to time constraint, still a work in progress :)

import scipy.io.wavfile as wav
import argparse
import torch.nn.functional as F
from torch.nn import init
import tensorflow as tf 
import numpy as np
import os
import sys
from sklearn.metrics import roc_curve, auc
sys.path.append("DeepSpeech")
from os.path import exists, join, basename, splitext

!apt-get install -qq sox
!pip install -q deepspeech-gpu==0.6.1 youtube-dl
!wget https://github.com/mozilla/DeepSpeech/releases/download/v0.6.1/deepspeech-0.6.1-models.tar.gz
!tar xvfz deepspeech-0.6.1-models.tar.gz

!pip install util

import DeepSpeech

tf.load_op_library = lambda x: x
tmp = os.path.exists
os.path.exists = lambda x: True

os.path.exists = tmp

from util.text import ctc_label_dense_to_sparse
from tf_logits3 import get_logits
from tqdm import tqdm
toks = " abcdefghijklmnopqrstuvwxyz'-"

DeepSpeech.TrainingCoordinator.__init__ = lambda x: None

DeepSpeech.TrainingCoordinator.start = lambda x: None
import loss
# Use the same decode framework as the CW attack
class Decoder:
	def __init__(self, sess, max_audio_len):
		self.sess = sess
		self.max_audio_len = max_audio_len
		self.original = original = tf.Variable(np.zeros((1, max_audio_len), dtype=np.float32), name='qq_original')
		self.lengths = lengths = tf.Variable(np.zeros(1, dtype=np.int32), name='qq_lengths')

		with tf.variable_scope("", reuse=tf.AUTO_REUSE):
			logits, features = get_logits(original, lengths)

		self.logits = logits
		self.features = features
		saver = tf.train.Saver([x for x in tf.global_variables() if 'qq' not in x.name])
		saver.restore(sess, "models/session_dump")
		self.decoded, _ = tf.nn.ctc_beam_search_decoder(logits, lengths, merge_repeated=False, beam_width=1000)

	def transcribe(self, audio, lengths):
		sess = self.sess
		sess.run(self.original.assign(np.array(audio)))
		sess.run(self.lengths.assign((np.array(lengths)-1)//320))
		out, logits = sess.run((self.decoded, self.logits))
		chars = out[0].values
		res = np.zeros(out[0].dense_shape)+len(toks)-1		
		for ii in range(len(out[0].values)):
			x,y = out[0].indices[ii]
			res[x,y] = out[0].values[ii]
		res = ["".join(toks[int(x)] for x in y).replace("-","") for y in res]
		return res[0]





In [ ]:
def decode(audio, num):
	global maxlen
	global D
	global ref
	audios = [list(audio)]
	lengths = [int(maxlen * num)]
	audios = np.array(audios)
	res = D.transcribe(audios, lengths)
	return res

In [ ]:
num_samples = 50
y_test = np.zeros(num_samples * 2)
roc_auc = np.zeros(3)
TD = np.zeros((3, num_samples * 2), dtype = np.float32)
count = 0
if __name__ == '__main__':
	sess = tf.Session()
	parser = argparse.ArgumentParser(description = None)
	parser.add_argument('--cut', type = float, required = True)
	args = parser.parse_args()


	ratio = args.cut
	pbar = tqdm(range(num_samples), unit='steps', ascii = True)
	for epoch in pbar:
		x, y = wav.read("librisample" + str(epoch) + ".wav")
		z, w = wav.read("librifinal" + str(epoch) + ".wav")
		maxlen = len(y)

		D = Decoder(sess, maxlen)
		stry = decode(y, 1)
		strw = decode(w, 1)
		halfy = decode(y, ratio)
		halfw = decode(w, ratio)

		s1 = loss.newWER(stry, halfy)
		s2 = loss.newCER(stry, halfy)
		s3 = loss.lcp(stry, halfy)

		y_test[count] = 0
		TD[0][count] = float(s1)
		TD[1][count] = float(s2)
		TD[2][count] = float(s3)

		count += 1
		s1 = loss.newWER(strw, halfw)
		s2 = loss.newCER(strw, halfw)
		s3 = loss.lcp(strw, halfw)
		
		y_test[count] = 1
		TD[0][count] = float(s1)
		TD[1][count] = float(s2)
		TD[2][count] = float(s3)
		count += 1

	for i in range(3):
		if (i == 2):
			y_test = 1 - y_test
		fpr, tpr, threshold = roc_curve(y_test, TD[i])
		roc_auc[i] = auc(fpr, tpr)

	print ("WER: " + str(roc_auc[0]) + " CER: " + str(roc_auc[1]) + " LCP: " + str(roc_auc[2]))

In [ ]:
class AudioClassifier (nn.Module):
    def __init__(self):
        super().__init__()
        conv_layers = []
        self.conv1 = nn.Conv2d(2, 8, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
        self.relu1 = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(8)
        init.kaiming_normal_(self.conv1.weight, a=0.1)
        self.conv1.bias.data.zero_()
        conv_layers += [self.conv1, self.relu1, self.bn1]

        self.conv2 = nn.Conv2d(8, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.bn2 = nn.BatchNorm2d(16)
        init.kaiming_normal_(self.conv2.weight, a=0.1)
        self.conv2.bias.data.zero_()
        conv_layers += [self.conv2, self.relu2, self.bn2]

        self.ap = nn.AdaptiveAvgPool2d(output_size=1)
        self.lin = nn.Linear(in_features=64, out_features=10)

        # Wrap the Convolutional Blocks
        self.conv = nn.Sequential(*conv_layers)
 
    def forward(self, x):
        x = self.conv(x)
        x = self.ap(x)
        x = x.view(x.shape[0], -1)
        x = self.lin(x)
        return x

myModel = AudioClassifier()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
myModel = myModel.to(device)
next(myModel.parameters()).device

In [ ]:
#partial implemntation of anomalous pattern detection. Work in progress (a lot left to do and optimize)
def training(model, train_dl, num_epochs):
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
  scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=0.001,steps_per_epoch=int(len(train_dl)),epochs=num_epochs,anneal_strategy='linear')

  for epoch in range(num_epochs):
    running_loss = 0.0
    correct_prediction = 0
    total_prediction = 0

    for i, data in enumerate(train_dl):
        inputs, labels = data[0].to(device), data[1].to(device)

        inputs_m, inputs_s = inputs.mean(), inputs.std()
        inputs = (inputs - inputs_m) / inputs_s

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

        running_loss += loss.item()
        _, prediction = torch.max(outputs,1)
        correct_prediction += (prediction == labels).sum().item()
        total_prediction += prediction.shape[0]

    num_batches = len(train_dl)
    avg_loss = running_loss / num_batches
    acc = correct_prediction/total_prediction
    print(f'Epoch: {epoch}, Loss: {avg_loss:.2f}, Accuracy: {acc:.2f}')

  print('Finished Training')
  
num_epochs=2
training(myModel, train_dl, num_epochs)

def ExtractActivation(flattened_layer, input_data):

    return activations

def SortAscending(arr):
    return np.sort(arr)


def NPSS(alpha, k, m):
    return npss_score

def main(Xz, Xi, alpha_max, D_H0, J):
    M = len(D_H0)
    ASR = training(training_dataset)
    ASRy = SomeFlattenedLayerOfASR
    A_H0 = np.zeros((M, J))
    A_i = np.zeros(J)
    pij = np.zeros(M)
    psij = np.zeros(M)
    S = {}
    alpha = {}
    F = {}

  
    for z in range(M):
        for j in range(J):
            A_H0[z,j] = ExtractActivation(ASRy, Xz[z])

    for j in range(J):
        A_i[j] = ExtractActivation(ASRy, Xi)

    for i in range(M):
        if A_H0[i,j] >= A_i[j]:
            pij[i] += 1
        else:
            pij[i] += 0
    pij /= M
    pij += 1 / (M + 1)
    psij = SortAscending(pij)

    for k in range(1, J+1):
        S[k] = {}
        for y in range(k):
            S[k][y] = psij[y]
        alpha[k] = max(S[k].values())
        F[k] = NPSS(alpha[k], k, k)

    k_star = max(F, key=F.get)
    return S[k_star], alpha[k_star], F[k_star]

In [ ]:
def lcp(x, y):
	x = x.split()
	y = y.split()
	n = len(x)
	m = len(y)

	k = min(n, m)
	idx = 0
	for i in range(k):
		if (x[i] != y[i]):
			break
		idx = i + 1
	
	return idx * 1.0 / max(n, m)

In [ ]:
def newWER(x, y):
	x = x.split()
	y = y.split()
	
	n = len(x)
	m = len(y)
	k = min(n, m)
	d = np.zeros((k + 1) * (k + 1), dtype = np.uint8).reshape(k + 1, k + 1)

	for i in range(k + 1):
		for j in range(k + 1):
			if i == 0:
				d[0][j] = j
			elif j == 0:
				d[i][0] = i

	for i in range(1, k + 1):
		for j in range(1, k + 1):
			if (x[i - 1] == y[j - 1]):
				d[i][j] = d[i - 1][j - 1]
			else:
				S = d[i - 1][j - 1] + 1
				I = d[i][j - 1] + 1
				D = d[i - 1][j] + 1
				d[i][j] = min(S, I, D)
	
	print (d[k][k])
	return d[k][k] * 1.0 / k

In [ ]:
def WER(x, y):
	x = x.split()
	y = y.split()
	
	n = len(x)
	m = len(y)
	print (n)
	print (m)
	d = np.zeros((n + 1) * (m + 1), dtype = np.uint8).reshape(n + 1, m + 1)

	for i in range(n + 1):
		for j in range(m + 1):
			if i == 0:
				d[0][j] = j
			elif j == 0:
				d[i][0] = i

	for i in range(1, n + 1):
		for j in range(1, m + 1):
			if (x[i - 1] == y[j - 1]):
				d[i][j] = d[i - 1][j - 1]
			else:
				S = d[i - 1][j - 1] + 1
				I = d[i][j - 1] + 1
				D = d[i - 1][j] + 1
				d[i][j] = min(S, I, D)
	
	print (d[n][m])
	return d[n][m] * 1.0 / n




In [ ]:

def newCER(x, y):
	x = x.replace(" ", "")
	y = y.replace(" ", "")
	n = len(x)
	m = len(y)
	k = min(n, m)
	d = np.zeros((k + 1) * (k + 1), dtype = np.uint8).reshape(k + 1, k + 1)

	for i in range(k + 1):
		for j in range(k + 1):
			if i == 0:
				d[0][j] = j
			elif j == 0:
				d[i][0] = i

	for i in range(1, k + 1):
		for j in range(1, k + 1):
			if (x[i - 1] == y[j - 1]):
				d[i][j] = d[i - 1][j - 1]
			else:
				S = d[i - 1][j - 1] + 1
				I = d[i][j - 1] + 1
				D = d[i - 1][j] + 1
				d[i][j] = min(S, I, D)
	
	return d[k][k] * 1.0 / k


In [ ]:
def CER(x, y):
	x = x.replace(" ", "")
	y = y.replace(" ", "")
	n = len(x)
	m = len(y)
	d = np.zeros((n + 1) * (m + 1), dtype = np.uint8).reshape(n + 1, m + 1)

	for i in range(n + 1):
		for j in range(m + 1):
			if i == 0:
				d[0][j] = j
			elif j == 0:
				d[i][0] = i

	for i in range(1, n + 1):
		for j in range(1, m + 1):
			if (x[i - 1] == y[j - 1]):
				d[i][j] = d[i - 1][j - 1]
			else:
				S = d[i - 1][j - 1] + 1
				I = d[i][j - 1] + 1
				D = d[i - 1][j] + 1
				d[i][j] = min(S, I, D)
	
	return d[n][m] * 1.0 / n